In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load the data and preprocess it
df = pd.read_csv('/content/drive/MyDrive/Thesis2024/fer2013.csv')
df = df[(df['emotion'] != 1) & (df['emotion'] != 5)]  # Remove disgust and surprise emotions

pixels = df['pixels'].tolist()
X = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in pixels])
X = X.reshape(-1, 48, 48, 1) / 255.0  # Normalize pixel values
y = df['emotion'].values

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = pd.get_dummies(y).values  # Convert to one-hot encoding

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))  # Additional dense layer
model.add(Dense(5, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy on test set: {accuracy * 100:.2f}%')

# Save the model architecture as a JSON file
model_json = model.to_json()
with open('/content/drive/MyDrive/Thesis2024/model.json', 'w') as json_file:
    json_file.write(model_json)


Epoch 1/50
196/196 [==============================] - 4s 13ms/step - loss: 1.5835 - accuracy: 0.2850 - val_loss: 1.5640 - val_accuracy: 0.2902
Epoch 2/50
196/196 [==============================] - 2s 13ms/step - loss: 1.5212 - accuracy: 0.3404 - val_loss: 1.4506 - val_accuracy: 0.3958
Epoch 3/50
196/196 [==============================] - 3s 14ms/step - loss: 1.4252 - accuracy: 0.4080 - val_loss: 1.3797 - val_accuracy: 0.4355
Epoch 4/50
196/196 [==============================] - 2s 12ms/step - loss: 1.3605 - accuracy: 0.4389 - val_loss: 1.3322 - val_accuracy: 0.4588
Epoch 5/50
196/196 [==============================] - 2s 11ms/step - loss: 1.3142 - accuracy: 0.4606 - val_loss: 1.2862 - val_accuracy: 0.4757
Epoch 6/50
196/196 [==============================] - 2s 12ms/step - loss: 1.2749 - accuracy: 0.4759 - val_loss: 1.2556 - val_accuracy: 0.4871
Epoch 7/50
196/196 [==============================] - 2s 12ms/step - loss: 1.2411 - accuracy: 0.4914 - val_loss: 1.2234 - val_accuracy: 0.5034